# Customize your case: Parameter modifications
## *Customizing your case with namelist changes & spinning up the model*

**This is an optional tutorial** that provides some additional guidance for creating an experimental clone for a NEON case in CTSM and spinning up the model.

We're going to move kind of quickly through some material that's already covered in the tutorial on [cloning a NEON case](./1c_NEON_Clone_Case.ipynb).  We're also going to quickly overview spinning up the model, but there's additional information in the tutorial on [customizing model features](./customizeCase_modelFeatures.ipynb)It may be helpful to refer back to these tutorials if you have questions or want more information about creating an experimental clone or spinup.

Finally, this tutorial assumes that you've modified the CLM parameter file.  We provided an example notebook for [modifying parameters](./modifyParameterFile.ipynb).  **This should be done before running the rest of this notebook.**
<br>

---
## In this tutorial

1. Create a new base case for a NEON site that you've already run.
2. Point to a modified parameter file in user_nl_clm.
3. Spinup the model. 

---
### First let's start with a bit of background.

Parameter files are read into CLM.  We provide a default parameter file for your cases, but this can be modified by pointing to a new parameter file in user_nl_clm.  

Since changing parameters in the model will change answers, we also need to spin up the model to generate new initial conditions files.

<div class="alert alert-block alert-warning">

<b>NOTICE:</b> If you're running this notebook through the NCAR JupyterHub login, you need to be on a Cheyenne login node (NOT Casper).  

</div>

### Set up your environment
It is important in order to have all the tools and packages you need to run simulations. 

The following code **is needed** if you're running in CESM-lab in the cloud, but should already be set up correctly for you.

In [ ]:
#cp -rp /opt/ncar/ctsm/ ~/CTSM

*The code below **is not** needed in the cloud*.

<div class="alert alert-block alert-info">
   
<b>TIP:</b> <i>If you're running on Cheyenne</i>, you may need to uncomment the the following two lines of code. This will set up your conda environment correctly.

<b>This is not required if your running CESM-Lab in the cloud.</b>

</div>

In [ ]:
#module purge
#module load conda ncarenv

# 1. Create a new case.
Because we're going to modify namelist options that will change answers in CLM we have to run this case through an accelerated decompostion (AD) and postAD spinup.  This will generate new initial conditions for our experimental case. We'll learn more about spinup in part 3 of this tutorial.

Doing an AD spinup requires a new base case to be created so the model is configured correctly.  This means we have to create and build a new case for both AD and postAD simulations.  This process is streamlined with the `run_neon` script and the use of usermod directories we created for NEON cases.  

## 1.1 Navigate to your source code

In [ ]:
cd ~/CTSM/tools/site_and_regional

## 1.2 Create a new base case.
You can modify the new case for the site you'd like to run.

In [ ]:
# If needed, change the 4-character NEON site below.
export new_case='HARV'    # the new site you want to run 
export experiment='foliarCN-30'    # this is name of your experimenal case 

Then run_neon with the following flags:
- `--experiment $experiment` 
- `--run-type ad` 
- `--setup-only` 

Since we're creating an experimental case that will change answers in the model we should run a full AD and postAD spinup.
We'll go over specifics related to experiment and run-type later in this tutorial.

In our experimental case we'll be using the BTRAN formulation of soil moisture stress. 

<div class="alert alert-block alert-info">

<b>HINT:</b> It's helpful to have descriptive experiment names. 

</div>

In [ ]:
# then run_neon with experiment and setup-only flags
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --overwrite \
   --experiment $experiment \
   --run-type ad \
   --setup-only  

### 1.2.1 Build the base case
You may have noticed a warning in the code block above.  We'll go ahead and build the new base case above.  

*As before this takes some time so be patient.*

In [ ]:
cd ~/scratch/NEON_cases/$new_case

In [ ]:
qcmd -- ./case.build

### 1.2.2 Overwrite the *site.experiment.ad* case, now using the correct build
- move back to your source code
- run_neon again 

In [ ]:
cd ~/CTSM/tools/site_and_regional

In [ ]:
cd ~/CTSM/tools/site_and_regional

./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$new_case \
   --overwrite \
   --experiment $experiment \
   --run-type ad \
   --setup-only  

You may notice the following was printed from creating an experimental clone
```
WARNING: CLM is starting up from a cold state
```
This is good, since we're going to be starting the model from a cold state, or bare ground with the AD spinup.

### 1.2.3 Move to the case directory

In [ ]:
cd ~/scratch/NEON_cases/$new_case.$experiment.ad

# 2 Customizing your case: Namelist changes
The configuration of CLM can be customized via namelist modifications. 

These changes made using namelist files like `user_nl_clm`

## 2.1 Getting familiar with namelist files
`user_nl_<model>` files are created in the case directory after setting up your case with `./case.setup`
  - Note: This was already done for us since we cloned a case with run_neon
    
#### Take a look!
  - What other user `user_nl_<model>` files do you already have in this case directory?
  - What kind of information is already included in a `user_nl_clm` file?

We can explore these files on the comand line using an editor, by opening them directly in jupyter hub, or by simply printing the contents of the files to the screen here.

In [ ]:
ls user_nl_*

In [ ]:
cat user_nl_clm | tail -18

The compset that you created your case with sets up initial, or default namelist options. These can be found in `CaseDocs/lnd_in`

<style> 
table td, table th, table tr {text-align:left !important;}
</style>
<div class="alert alert-block alert-warning">
<b>Important:</b>   Don’t modify the lnd_in namelist file directly. Instead, make changes in user_nl_clm.
</div>

## 2.2 Looking deeper at namelist options 
All CLM namelist options are defined in the `lnd_in` file

We can explore this file on the comand line using an editor, by opening it directly in jupyter hub, or by simply printing the contents of the files to the screen here.  
- **NOTE:** This file is huge, so we'll just look at the first 65 lines

In [ ]:
cat CaseDocs/lnd_in | head -40

There's a lot to dig into here! So we'll just stick with a few highlights.

#### Take a look
**See what printed to in the cell above and answer the following questions**
- `finidat`: What initial conditions file are you using? 
- `fsurdat`: What surface dataset are you using?
- `paramfile`: What parameter file are you using?
- `spinup_state`: What is your spinup state?

Does this make sense?  Answers are in the hidden cell below.


- `finidat = ' '`  There are no initial conditions because this is an AD run
- `fsurdat = ".../surfdata_1x1_NEON_WOOD_hist_78pfts_CMIP6_simyr2000_c230111.nc"` This the surface dataset that's been modified for our NEON site.
- `paramfile = '.../ctsm51_params.c211112.nc'` is the default parameter file for CTSM5.1  
- `spinup_state = 2`, which is used for accelerated decomposition, or AD mode


***

- Additional information about customizing CTSM's configurations, including namelist modifications, are available in the [CTSM users guide](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/customizing-the-clm-configuration.html?highlight=namelist)
Namelist changes can also be used to modify variables on history file output. Simple modification of history file output DOES NOT require spinning up the model.
- A list of all the [CTSM history fields are available here](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/master_list_nofates.html)
- A list of all the [CTSM-FATES history fields are available here](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/master_list_fates.html)

## 2.3 Point to your modified parameter file

For this experiment we're changing the foliar C:N ratio for temperate deciduous forests.  Do do this we have to point to our modified parameter file. We can change user_nl_clm using an editor, by opening it directly in jupyter hub, or with the following comand.

In [ ]:
# This is the path to the modified parameter file you created

# If you're running on cheyenne, uncomment the line below
#echo "paramfile = '/glade/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

# If you're running with CESM-Lab in the cloud, uncomment the line below
echo "paramfile = '/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

Let make sure this worked.  We can:  
- preview namelists;
- check the  `lnd_in` file.

In [ ]:
## now make sure it worked 
./preview_namelists

In [ ]:
cat CaseDocs/lnd_in | head -40

We've changes ethe parameter file that's being used for our simulation.

## 2.4 Start the AD spinup run
We already build the model, and when making namelist or .xml changes we don't have to rebuild our case, so now we're ready to submit the case.

In [ ]:
./case.submit

Now our case is submitted!  Spinup, or model initialization from a cold start, actually takes a little bit of time.  Check that your AD case is running 

In [ ]:
qstat -u $USER

In [ ]:
tail  ~/scratch/NEON_cases/$new_case.$experiment.ad/CaseStatus

## 3.1 postAD simulations
After the model has reached steady-state we have to take it out of AD mode, which requires another ~100 years of simulations.  We also have to create and build a new base case for postAD runs.

### 3.1.1 Create a new base case

In [ ]:
cd ~/CTSM/tools/site_and_regional
# This is a new base case that we'll create (and maybe use later for other experiments)
export base_case='YELL'    
# should match the neon site case you did the AD run with
export new_case='HARV'   

In [ ]:
./run_neon.py --neon-sites $base_case  \
   --output-root ~/scratch/NEON_cases \
   --overwrite \
   --run-type postad \
   --setup-only 

You'll notice an error here if there's no ad run for our postad base_case.  This is fine. 

### 3.1.2 Build the base case

In [ ]:
cd ~/scratch/NEON_cases/$base_case
qcmd -- ./case.build

### 3.1.3 Create the site.experiment.postad case from the built base case

In [ ]:
cd ~/CTSM/tools/site_and_regional

./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$base_case \
   --overwrite \
   --experiment $experiment \
   --run-type postad \
   --setup-only  

### 3.1.4 make the same namelist changes to point to your new parameter file.

In [ ]:
cd ~/scratch/NEON_cases/$new_case.$experiment.postad

# This is the path to the modified parameter file you created

# If you're running on cheyenne, uncomment the line below
#echo "paramfile = '/glade/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

# If you're running with CESM-Lab in the cloud, uncomment the line below
echo "paramfile = '/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

./preview_namelists

In [ ]:
# check the lnd_in file
cat CaseDocs/lnd_in | grep paramfile

**Stop and check!**
- How are the `lnd_in` files different in our ad and postad cases?

In [ ]:
diff CaseDocs/lnd_in ../$new_case.$experiment.ad/CaseDocs/lnd_in

the postad case has: 
- `finidat = 'HARV.foliarCN-30.ad...'` initial conditions
- `spinup_state = 0` meaning we're out of ad mode now
- different builds


### 3.1.5 Submit the postad simulation

In [ ]:
./case.submit

## 3.2 Transient simulations
Now we're ready to go ahead with the transient simulation

### 3.2.1 Create an experimental transient case
- your `base_case` can now be a built transient case (here KONZ from our first tutorials)
- `--run-from-postad` let's us get initial conditions files from our experiment.postad case

In [ ]:
# Change the 4-character NEON site below.
cd ~/CTSM/tools/site_and_regional
export base_case='KONZ'   # this should be a built, transient case
export new_case='HARV'    # the new site you want to run

# then run_neon
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$base_case \
   --experiment $experiment \
   --overwrite \
   --run-type transient \
   --setup-only  \
   --run-from-postad


### 3.2.2 Namelist changes to point to your new parameter file

In [ ]:
cd ~/scratch/NEON_cases/$new_case.$experiment.transient

# This is the path to the modified parameter file you created

# If you're running on cheyenne, uncomment the line below
#echo "paramfile = '/glade/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

# If you're running with CESM-Lab in the cloud, uncomment the line below
echo "paramfile = '/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

./preview_namelists

In [ ]:
# check the lnd_in file
cat CaseDocs/lnd_in | grep paramfile

In [ ]:
# How are the lnd_in fies different?
diff CaseDocs/lnd_in ../$new_case.$experiment.postad/CaseDocs/lnd_in

Now we're running: 
- a transient simulation (transient CO2, nitrogen deposition, etc)
- with different restart files, not from the postad run that are prestaged from default NEON cases
- with different builds

### 3.2.3 Submit the transient case

In [ ]:
./case.submit

## 3.3 Run a control case
If you haven't done it already, we should likely just run a control case for a transient case out of the box (with PHS on).
Lucking this is really easy!

In [ ]:
cd ~/CTSM/tools/site_and_regional

# then run_neon
./run_neon.py --neon-sites $new_case  \
           --output-root ~/scratch/NEON_cases \
           --base-case ~/scratch/NEON_cases/$base_case \
           --overwrite

# we'll just go ahead and submit this case

#### Double check
It's a good idea to make sure cases have expected differences.  

See if this is true for our control and experimental transient cases.

In [ ]:
cd ~/scratch/NEON_cases/$new_case.transient
diff CaseDocs/lnd_in ../$new_case.$experiment.transient/CaseDocs/lnd_in

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created and run a full spinup for an experimental case of CLM at the NEON tower you selected!
    
Now we need to see how the simulations are different.
</div>



If you're developing this tutorial:
Before saving and pushing this code to github go to `Kernel` and `Restart kernel and clear all outputs...`